In [ ]:
# !pip install transformers

import pandas as pd
import numpy as np
import re
from pathlib import Path
from sklearn.model_selection import train_test_split


import torch
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

## Data

In [ ]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll

In [ ]:
# Read a CoNLL file

def read_conll(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_conll('wnut17train.conll')

In [ ]:
# Encode tag and texts

unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
# Train test split

train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [ ]:
# Tokenize text

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
# Handle out-of-vocabulary words by only training on the tag labels for the first subtoken of a split token and setting the labels we wish to ignore to -100


def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
# Dataset Class

class WNUTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

## Train

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

In [ ]:
# Train the model

idx = 0
model_path = f'gdrive/MyDrive/model_{idx}'

training_args = TrainingArguments(
    output_dir=model_path,          # output directory
    num_train_epochs=1, 
    evaluation_strategy="epoch"             # total number of training epochs
)

# Trainer object 

trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=train_dataset,        
    eval_dataset=val_dataset             
)

trainer.train()

## Evaluate

In [ ]:
trainer.evaluate()

## Save

In [ ]:
!mkdir ner_model
trainer.save_model("ner_model")

## Predict

In [ ]:
test_model = DistilBertForTokenClassification.from_pretrained("ner_model")

In [ ]:
test_sentence = "Huggingface is an organization in New York"
test_tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(test_sentence)))
test_encodings = tokenizer.encode(test_sentence, return_tensors="pt")

In [ ]:
pred = test_model(test_encodings)[0]

In [ ]:
label_list = list(unique_tags)
pred = torch.argmax(pred, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(test_tokens, pred[0].tolist())])